# Mô hình dự báo giá Bitcoin với BiLSTM

Notebook này hướng dẫn từng bước để xây dựng và huấn luyện mô hình dự báo giá Bitcoin.

## Setup & Cấu hình

Cấu hình TensorFlow và import các thư viện cần thiết.

In [ ]:
# Standard library imports
import logging
import os
import sys
import warnings
from pathlib import Path

# Suppress TensorFlow warnings BEFORE importing TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Chỉ hiển thị ERROR
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Tắt oneDNN warnings

# Suppress Python warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', message='.*np.object.*')
warnings.filterwarnings('ignore', message='.*oneDNN.*')
warnings.filterwarnings('ignore', message='.*CUDA.*')
warnings.filterwarnings('ignore', message='.*Could not find cuda.*')
warnings.filterwarnings('ignore', message='.*cuda drivers.*')
warnings.filterwarnings('ignore', message='.*GPU will not be used.*')

# Third-party imports (must be after env vars for TensorFlow)
import numpy as np  # noqa: E402
import polars as pl  # noqa: E402

# Import và suppress TensorFlow logger ngay lập tức
try:
    import tensorflow as tf  # noqa: E402
    tf.get_logger().setLevel('ERROR')
    # Suppress stderr output từ TensorFlow
    logging.getLogger('tensorflow').setLevel(logging.ERROR)
except ImportError:
    pass  # TensorFlow chưa được cài đặt

# Setup project path (notebook runs from notebooks/, need to go up one level)
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Local imports
from src.runtime import (  # noqa: E402
    configure_tensorflow_runtime,
    print_tensorflow_info,
    set_random_seed
)

# Cấu hình TensorFlow cho CPU AMD
configure_tensorflow_runtime(
    intra_op_threads=12,
    inter_op_threads=2,
    enable_xla=True
)

# In thông tin TensorFlow
print_tensorflow_info()

## Chọn Preset

Chọn một trong các preset có sẵn hoặc cấu hình thủ công:

In [ ]:
# ==================== CHỌN PRESET ====================
# Chọn preset từ danh sách bên dưới, hoặc set PRESET_NAME = None để cấu hình thủ công
#
# Preset Scalping (15m - siêu ngắn hạn):
# - 'scalping-ultra-fast': 10k dòng, window=24 (6h), epochs=5, lstm=[16], dense=[] - Scalping cực nhanh
# - 'scalping-fast': 20k dòng, window=48 (12h), epochs=10, lstm=[32,16], dense=[16] - Scalping nhanh
#
# Preset Intraday (15m - ngắn hạn):
# - 'intraday-light': 30k dòng, window=96 (1d), epochs=15, lstm=[32,16], dense=[16] - Intraday nhẹ
# - 'intraday-balanced': 50k dòng, window=144 (1.5d), epochs=25, lstm=[64,32], dense=[32] - Intraday cân bằng
#
# Preset Swing (15m - trung hạn):
# - 'swing-fast': 70k dòng, window=240 (2.5d), epochs=30, lstm=[64,32], dense=[32] - Swing nhanh
# - 'swing-balanced': 100k dòng, window=384 (4d), epochs=50, lstm=[128,64,32], dense=[64,32] - Swing cân bằng
#
# Preset Long-term & Production (15m):
# - 'long-term': 150k dòng, window=576 (6d), epochs=80, lstm=[256,128,64,32], dense=[128,64] - Dự đoán dài hạn
# - 'production': 200k dòng, window=768 (8d), epochs=100, lstm=[256,128,64,32], dense=[128,64,32] - Production chất lượng cao nhất
#
# Preset 30k Dataset (15m - Window size từ ngắn đến dài hạn, intra_op_threads=12):
# - '30k-w24': 30k dòng, window=24 (6h), epochs=15, lstm=[32,16], dense=[16] - Ngắn hạn cực nhanh
# - '30k-w48': 30k dòng, window=48 (12h), epochs=15, lstm=[32,16], dense=[16] - Ngắn hạn nhanh
# - '30k-w72': 30k dòng, window=72 (18h), epochs=20, lstm=[32,16], dense=[16] - Ngắn hạn
# - '30k-w96': 30k dòng, window=96 (1d), epochs=20, lstm=[64,32], dense=[32] - Ngắn hạn cân bằng (kết quả tốt)
# - '30k-w144': 30k dòng, window=144 (1.5d), epochs=25, lstm=[64,32], dense=[32] - Trung hạn ngắn
# - '30k-w192': 30k dòng, window=192 (2d), epochs=25, lstm=[64,32], dense=[32] - Trung hạn
# - '30k-w240': 30k dòng, window=240 (2.5d), epochs=30, lstm=[64,32], dense=[32] - Trung hạn cân bằng
# - '30k-w336': 30k dòng, window=336 (3.5d), epochs=30, lstm=[128,64,32], dense=[64,32] - Trung hạn dài
# - '30k-w480': 30k dòng, window=480 (5d), epochs=40, lstm=[128,64,32], dense=[64,32] - Dài hạn ngắn
# - '30k-w672': 30k dòng, window=672 (7d), epochs=40, lstm=[128,64,32], dense=[64,32] - Dài hạn
#
# Preset Legacy (cho các timeframe khác):
# - 'default': 50k dòng 15m (default config)
# - 'fast': 20k dòng 15m (test nhanh)
# - '1h-light': 10k dòng 1h
# - '4h-balanced': 2k dòng 4h
#
# Set PRESET_NAME = None để dùng cấu hình thủ công ở cell bên dưới

PRESET_NAME = '30k-w144'  # Đổi tên preset ở đây, hoặc None để cấu hình thủ công

# Import preset functions
from src import (  # noqa: E402
    Config,
    # Scalping presets
    get_scalping_ultra_fast_config,
    get_scalping_fast_config,
    # Intraday presets
    get_intraday_light_config,
    get_intraday_balanced_config,
    # Swing presets
    get_swing_fast_config,
    get_swing_balanced_config,
    # Long-term & Production presets
    get_long_term_config,
    get_production_config,
    # 30k dataset presets
    get_30k_w24_config,
    get_30k_w48_config,
    get_30k_w72_config,
    get_30k_w96_config,
    get_30k_w144_config,
    get_30k_w192_config,
    get_30k_w240_config,
    get_30k_w336_config,
    get_30k_w480_config,
    get_30k_w672_config,
    # Legacy presets
    get_default_config,
    get_fast_config,
    get_1h_light_config,
    get_4h_balanced_config,
)

# Map preset name với function
PRESET_MAP = {
    # Scalping presets
    'scalping-ultra-fast': get_scalping_ultra_fast_config,
    'scalping-fast': get_scalping_fast_config,
    # Intraday presets
    'intraday-light': get_intraday_light_config,
    'intraday-balanced': get_intraday_balanced_config,
    # Swing presets
    'swing-fast': get_swing_fast_config,
    'swing-balanced': get_swing_balanced_config,
    # Long-term & Production presets
    'long-term': get_long_term_config,
    'production': get_production_config,
    # 30k dataset presets
    '30k-w24': get_30k_w24_config,
    '30k-w48': get_30k_w48_config,
    '30k-w72': get_30k_w72_config,
    '30k-w96': get_30k_w96_config,
    '30k-w144': get_30k_w144_config,
    '30k-w192': get_30k_w192_config,
    '30k-w240': get_30k_w240_config,
    '30k-w336': get_30k_w336_config,
    '30k-w480': get_30k_w480_config,
    '30k-w672': get_30k_w672_config,
    # Legacy presets
    'default': get_default_config,
    'fast': get_fast_config,
    '1h-light': get_1h_light_config,
    '4h-balanced': get_4h_balanced_config,
}

# Load preset hoặc tạo config rỗng
if PRESET_NAME and PRESET_NAME in PRESET_MAP:
    config = PRESET_MAP[PRESET_NAME]()
    print(f"Đã chọn preset: {PRESET_NAME}")
    print(f"   Limit: {config.data.limit}, Window: {config.preprocessing.window_size}, Epochs: {config.training.epochs}")
    print(f"   LSTM units: {config.model.lstm_units}\n")
else:
    config = Config()
    if PRESET_NAME:
        print(f"Không tìm thấy preset '{PRESET_NAME}', dùng cấu hình mặc định.\n")
    else:
        print("Không chọn preset, sẽ dùng cấu hình thủ công ở cell bên dưới.\n")

### Tùy chỉnh thêm (Ghi đè preset hoặc cấu hình thủ công)

In [ ]:
# ==================== CẤU HÌNH THỦ CÔNG / GHI ĐỀ PRESET ====================

# NOTE: Nếu đã chọn preset ở cell trên, các giá trị None sẽ dùng preset.
#       Set giá trị cụ thể để ghi đè preset.

# Reproducibility
SEED = config.runtime.seed  # Đổi seed để chạy thử nghiệm; đặt <0 để không cố định

# Symbol
SYMBOL = "BTC/USDT"  # Trading pair symbol

# Data parameters (CSV local)
# Sử dụng project_root để tìm đường dẫn đúng (notebook chạy từ notebooks/)
try:
    # project_root đã được định nghĩa trong cell 2
    data_file_relative = "data/btc_15m_data_2018_to_2025.csv"  # Đổi sang 4h/1h/15m: btc_4h_data_2018_to_2025.csv, btc_1h_data_2018_to_2025.csv, btc_15m_data_2018_to_2025.csv
    DATA_PATH = str(project_root / data_file_relative)
except NameError:
    # Fallback nếu project_root chưa được định nghĩa (chạy cell này trước cell 2)
    project_root = Path.cwd().parent
    data_file_relative = "data/btc_15m_data_2018_to_2025.csv"
    DATA_PATH = str(project_root / data_file_relative)

LIMIT = None  # Lấy N dòng cuối (<=0 = lấy tất cả), None = dùng preset
REFRESH_CACHE = False  # True = đọc lại CSV gốc, False = dùng cache normalized


def infer_timeframe_from_filename(path: str) -> str:
    """Infer timeframe from filename."""
    name = path.lower()
    # Thử theo thứ tự từ dài đến ngắn để tránh ghi đè không chính xác
    if "15m" in name:
        return "15m"
    if "1h" in name:
        return "1h"
    if "4h" in name:
        return "4h"
    if "1d" in name:
        return "1d"
    return "unknown"


TIMEFRAME = infer_timeframe_from_filename(DATA_PATH)  # Hiển thị timeframe dựa vào tên file

# Preprocessing parameters (None = dùng preset)
WINDOW_SIZE = None  # Số nến nhìn lại (sliding window)
FEATURES = ["close"]  # Features sử dụng

# Model parameters (None = dùng preset)
LSTM_UNITS = None  # Số units cho mỗi LSTM layer
DROPOUT_RATE = None  # Dropout rate

# Training parameters (None = dùng preset)
EPOCHS = None  # Số epochs
BATCH_SIZE = None  # Batch size
EARLY_STOPPING_PATIENCE = None  # Số epochs chờ trước khi dừng

# Ghi đè config với các giá trị đã set
if SEED is not None:
    config.runtime.seed = SEED
if LIMIT is not None:
    config.data.limit = LIMIT
if TIMEFRAME:
    config.data.timeframe = TIMEFRAME
if DATA_PATH:
    config.data.data_path = DATA_PATH
if REFRESH_CACHE is not None:
    config.data.refresh_cache = REFRESH_CACHE
if FEATURES is not None:
    config.data.features = FEATURES
if WINDOW_SIZE is not None:
    config.preprocessing.window_size = WINDOW_SIZE
if LSTM_UNITS is not None:
    config.model.lstm_units = LSTM_UNITS
if DROPOUT_RATE is not None:
    config.model.dropout_rate = DROPOUT_RATE
if EPOCHS is not None:
    config.training.epochs = EPOCHS
if BATCH_SIZE is not None:
    config.training.batch_size = BATCH_SIZE
if EARLY_STOPPING_PATIENCE is not None:
    config.training.early_stopping_patience = EARLY_STOPPING_PATIENCE

# Cố định seed để tái lập kết quả (nếu SEED < 0 thì bỏ qua)
if config.runtime.seed is not None and config.runtime.seed >= 0:
    set_random_seed(config.runtime.seed, deterministic=True)

# Print cấu hình
print("=" * 60)
print("CẤU HÌNH CUỐI CÙNG")
print("=" * 60)
print(f"Seed: {config.runtime.seed}")
print(f"Data path: {DATA_PATH}")
print(f"Timeframe: {TIMEFRAME}")
print(f"Limit: {config.data.limit}")
print(f"Window size: {config.preprocessing.window_size}")
print(f"Features: {config.data.features}")
print(f"LSTM units: {config.model.lstm_units}")
print(f"Dropout: {config.model.dropout_rate}")
print(f"Epochs: {config.training.epochs}")
print(f"Batch size: {config.training.batch_size}")
print("=" * 60 + "\n")

---

## BƯỚC 1: ĐỌC DỮ LIỆU CSV (LOCAL)

### Giải thích:
- Đọc dữ liệu giá từ file CSV local (mặc định: `data/btc_15m_data_2018_to_2025.csv`)
- Chuẩn hoá về DataFrame với: datetime, open, high, low, close, volume
- Cache (optional) file CSV đã chuẩn hoá để lần sau đọc nhanh hơn

In [ ]:
from src.core import fetch_binance_data  # noqa: E402

# Đọc dữ liệu (CSV local)
df = fetch_binance_data(
    data_path=DATA_PATH,
    timeframe=TIMEFRAME,
    limit=config.data.limit,
    save_cache=not REFRESH_CACHE
)

# In 5 dòng đầu tiên
print("\n5 dòng đầu tiên của dữ liệu:")
print(df.head())

# Thống kê cơ bản
print("\nThống kê dữ liệu:")
print(df.describe())

# Đánh dấu checklist
print("\nBước 1 hoàn thành!")

### Vẽ biểu đồ lịch sử giá

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

plt.figure(figsize=(14, 6))
plt.plot(df['datetime'], df['close'], linewidth=2, color='#2E86AB', label='Giá đóng nến')
plt.title(f'Lịch sử giá Bitcoin ({TIMEFRAME})', fontsize=16, fontweight='bold')
plt.xlabel('Thời gian', fontsize=12)
plt.ylabel('Giá (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
plt.tight_layout()
plt.show()

---

## BƯỚC 2: XỬ LÝ DỮ LIỆU

### Giải thích:
- **Scaling**: Đưa dữ liệu về khoảng [0, 1] để model học tốt hơn
- **Chống data leakage**: scaler được **fit chỉ trên tập train**, sau đó mới transform val/test
- **Sliding Window**: Tạo sequences (window_size nến trước → dự đoán nến tiếp theo)
- **Split Data**: Chia thành train (70%), val (15%), test (15%)

In [ ]:
from src.core import prepare_data_for_lstm  # noqa: E402

# Pipeline xử lý dữ liệu hoàn chỉnh
data_dict = prepare_data_for_lstm(
    df=df,
    features=config.data.features,
    window_size=config.preprocessing.window_size,
    scaler_type='minmax'
)

# Lấy các biến
X_train = data_dict['X_train']
y_train = data_dict['y_train']
X_val = data_dict['X_val']
y_val = data_dict['y_val']
X_test = data_dict['X_test']
y_test = data_dict['y_test']
scaler = data_dict['scaler']

# Đánh dấu checklist
print("\nBước 2 hoàn thành!")

### Kiểm tra shapes của dữ liệu

In [ ]:
print("\n" + "="*60)
print("SHAPES CỦA DỮ LIỆU")
print("="*60)
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val:   {X_val.shape}")
print(f"y_val:   {y_val.shape}")
print(f"X_test:  {X_test.shape}")
print(f"y_test:  {y_test.shape}")
print("="*60 + "\n")

---

## BƯỚC 3: XÂY DỰNG MODEL BiLSTM

### Giải thích:
- **BiLSTM**: LSTM hai chiều (nhìn cả quá khứ và tương lai)
- **Dropout**: Bỏ ngẫu nhiên neurons để tránh overfitting
- **Dense layers**: Kết hợp features để đưa ra dự đoán

In [ ]:
from src.core import build_bilstm_model, print_model_summary  # noqa: E402

# Xây dựng model
input_shape = (config.preprocessing.window_size, len(config.data.features))
model = build_bilstm_model(
    input_shape=input_shape,
    lstm_units=config.model.lstm_units,
    dropout_rate=config.model.dropout_rate,
    dense_units=config.model.dense_units,
    output_units=1
)

# In thông tin model
print_model_summary(model)

# Đánh dấu checklist
print("\nBước 3 hoàn thành!")

---

## BƯỚC 4: TRAINING MODEL

### Giải thích:
- **ModelCheckpoint**: Lưu lại model tốt nhất
- **EarlyStopping**: Dừng nếu val_loss không giảm
- **ReduceLROnPlateau**: Giảm learning rate nếu không cải thiện

In [ ]:
from src.training import train_model  # noqa: E402

# Training
train_result = train_model(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    config=config
)

# Lấy training history
history = train_result['history']

# Metadata training (để đưa vào report)
best_epoch = train_result.get('best_epoch')
best_val_loss = train_result.get('best_val_loss')
train_seconds = train_result.get('train_seconds')
checkpoint_path_raw = train_result.get('checkpoint_path')
checkpoint_path = str(checkpoint_path_raw) if checkpoint_path_raw is not None else None

print(f"\nBest epoch: {best_epoch}")
print(f"Best val_loss: {best_val_loss}")
print(f"Training time (s): {train_seconds}")

# Đánh dấu checklist
print("\nBước 4 hoàn thành!")

### Vẽ biểu đồ training history

In [ ]:
from src.visualization import plot_training_history  # noqa: E402

plot_training_history(history)

---

## BƯỚC 5: ĐÁNH GIÁ & VẼ BIỂU ĐỒ

### Giải thích:
- **MAE**: Sai số trung bình tuyệt đối (USD)
- **RMSE**: Căn bậc 2 của sai số bình phương trung bình (USD)
- **MAPE**: Sai số phần trăm trung bình (%)

In [ ]:
from src.core import (  # noqa: E402
    calculate_direction_accuracy,
    evaluate_model,
    print_sample_predictions
)
from src.visualization import plot_all_in_one, plot_predictions  # noqa: E402

# Đánh giá trên test set
eval_result = evaluate_model(
    model=model,
    X_test=X_test,
    y_test=y_test,
    scaler=scaler,
    return_predictions=True
)

# Lấy dự đoán và giá trị thật
y_true = eval_result['y_true']
y_pred = eval_result['predictions']

In [ ]:
# In một số ví dụ dự đoán
print_sample_predictions(y_true, y_pred, n_samples=10)

In [ ]:
# Tính độ chính xác xu hướng
direction_accuracy = calculate_direction_accuracy(y_true, y_pred)

# Lưu vào eval_result để report/metrics.json có thêm thông tin
eval_result['direction_accuracy'] = float(direction_accuracy)

### Vẽ biểu đồ predictions vs actual

In [ ]:
plot_predictions(y_true, y_pred)

### Vẽ biểu đồ tổng hợp (all-in-one)

In [ ]:
plot_all_in_one(history, y_true, y_pred)

---

## LƯU KẾT QUẢ

Tất cả kết quả được lưu vào thư mục `reports/notebook/`

In [ ]:
from src.results import (  # noqa: E402
    create_results_folder,
    save_config,
    save_markdown_report,
    save_metrics
)

# Tạo folder kết quả với config để đặt tên chuẩn hóa
folder_config = {
    'timeframe': TIMEFRAME,
    'window_size': config.preprocessing.window_size,
    'limit': config.data.limit
}
results_folder = create_results_folder(run_type="notebook", config=folder_config)
print(f"Folder kết quả: {results_folder}\n")

# Tạo suffix cho tên file (lấy timestamp từ tên folder: 2 phần cuối)
folder_parts = results_folder.name.split('_')
timestamp_suffix = '_'.join(folder_parts[-2:])  # Lấy 2 phần cuối (YYYYMMDD_HHMMSS)

# Vẽ và lưu biểu đồ
plot_history_file = results_folder / f"training_history_{timestamp_suffix}.png"
plot_predictions_file = results_folder / f"predictions_{timestamp_suffix}.png"
plot_all_in_one_file = results_folder / f"all_in_one_{timestamp_suffix}.png"

plot_training_history(history, save_path=str(plot_history_file))
plot_predictions(y_true, y_pred, save_path=str(plot_predictions_file))
plot_all_in_one(history, y_true, y_pred, save_path=str(plot_all_in_one_file))

# Metadata dữ liệu
try:
    data_rows = len(df)
    data_start = str(df.select('datetime').row(0)[0])
    data_end = str(df.select('datetime').row(-1)[0])
except Exception:
    data_rows = None
    data_start = None
    data_end = None

# Metadata split
train_samples = len(X_train)
val_samples = len(X_val)
test_samples = len(X_test)
scaler_type = 'minmax'

# Chuẩn bị config và metrics
config_dict = {
    'data_path': DATA_PATH,
    'symbol': SYMBOL,
    'timeframe': TIMEFRAME,
    'limit': config.data.limit,
    'data_rows': data_rows,
    'data_start': data_start,
    'data_end': data_end,
    'window_size': config.preprocessing.window_size,
    'features': config.data.features,
    'scaler_type': scaler_type,
    'train_samples': train_samples,
    'val_samples': val_samples,
    'test_samples': test_samples,
    'seed': config.runtime.seed,
    'lstm_units': config.model.lstm_units,
    'dropout_rate': config.model.dropout_rate,
    'epochs': config.training.epochs,
    'batch_size': config.training.batch_size,
    'early_stopping_patience': config.training.early_stopping_patience,
    'learning_rate': config.training.learning_rate,
    'intra_threads': config.runtime.intra_op_threads,
    'best_epoch': best_epoch,
    'best_val_loss': best_val_loss,
    'train_seconds': train_seconds,
    'checkpoint_path': checkpoint_path,
}

plots_dict = {
    'training_history': f"training_history_{timestamp_suffix}.png",
    'predictions': f"predictions_{timestamp_suffix}.png",
    'all_in_one': f"all_in_one_{timestamp_suffix}.png"
}

# Lưu báo cáo
save_markdown_report(
    folder_path=results_folder,
    config=config_dict,
    metrics=eval_result,
    history=history.history,
    plots=plots_dict
)
save_config(results_folder, config_dict)
save_metrics(results_folder, eval_result)

# Đánh dấu checklist
print("\nBước 5 hoàn thành!")

---

## HOÀN THÀNH

### Checklist:
- [x] Bước 0: Setup & Cấu hình
- [x] Bước 1: Đọc dữ liệu CSV (local)
- [x] Bước 2: Xử lý dữ liệu
- [x] Bước 3: Xây dựng model BiLSTM
- [x] Bước 4: Training model
- [x] Bước 5: Đánh giá & Vẽ biểu đồ
- [x] Bước 6: Lưu kết quả

### Kết quả:
- Báo cáo Markdown: `reports/notebook/BiLSTM_YYYYMMDD_HHMMSS/results_*.md`
- Biểu đồ: Các file PNG trong cùng folder
- Config & Metrics: File JSON trong cùng folder

### Tiếp theo:
- Thử thay đổi preset:
  - **Scalping (15m):** scalping-ultra-fast, scalping-fast
  - **Intraday (15m):** intraday-light, intraday-balanced
  - **Swing (15m):** swing-fast, swing-balanced
  - **Long-term (15m):** long-term
  - **Production (15m):** production
  - **30k Dataset (15m - Window size từ ngắn đến dài hạn, intra_op_threads=12):**
    - 30k-w24, 30k-w48, 30k-w72, 30k-w96, 30k-w144, 30k-w192, 30k-w240, 30k-w336, 30k-w480, 30k-w672
  - **Legacy (other timeframes):** default, fast, 1h-light, 4h-balanced
- Hoặc cấu hình thủ công bằng cách set PRESET_NAME = None
- Thêm các features khác (volume, open, high, low)
- Thử timeframe khác (1d, 4h, 1h, 15m) - default là 15m